# Подключение библиотек, загрузка и очистка датасета, кодирование категориальных признаков

In [31]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, KFold, cross_val_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from warnings import simplefilter

simplefilter('ignore')

In [4]:
data = pd.read_csv("sample_data/HousingData.csv")

In [5]:
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,NaN,36.2


## CRIM: средняя преступность на душу населения в городе.
## ZN: доля земли, зарезервированной под участки более 25 000 кв.футов.
## INDUS: доля неторговых бизнес-акров на город.
## CHAS: переменная Charles River (равна 1, если участок граничит с рекой; 0 в противном случае).
## NOX: концентрация нитритных оксидов (части на 10 миллионов).
## RM: среднее количество комнат в жилье.
## AGE: доля домов, построенных до 1940 года.
## DIS: взвешенные расстояния до пяти центров занятости в Бостоне.
## RAD: индекс доступности к радиальным автомагистралям.
## TAX: средний налоговый ставка на недвижимость на $10 000.
## PTRATIO: соотношение учеников к учителям в городе.
## B: 1000(Bk - 0.63)^2, где Bk - доля чернокожих в городе.
## LSTAT: процент населения с низким социальным статусом.
## MEDV: медианная стоимость дома с собственником в тысячах долларов

In [6]:
data.isna().sum()

CRIM       20
ZN         20
INDUS      20
CHAS       20
NOX         0
RM          0
AGE        20
DIS         0
RAD         0
TAX         0
PTRATIO     0
B           0
LSTAT      20
MEDV        0
dtype: int64

In [7]:
data.fillna(data.mean(), inplace=True)


In [8]:
data.isna().sum()


CRIM       0
ZN         0
INDUS      0
CHAS       0
NOX        0
RM         0
AGE        0
DIS        0
RAD        0
TAX        0
PTRATIO    0
B          0
LSTAT      0
MEDV       0
dtype: int64

## Разделение выборки на обучающую и на тестовую

In [9]:
y = data['MEDV']
X = data.drop('MEDV', axis=1)
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=3)
x_train

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
359,4.26131,0.0,11.083992,0.000000,0.770,6.112,81.3,2.5091,24,666,20.2,390.74,12.670000
374,18.49820,0.0,18.100000,0.000000,0.668,4.138,100.0,1.1370,24,666,20.2,396.90,37.970000
98,0.08187,0.0,2.890000,0.000000,0.445,7.820,36.9,3.4952,2,276,18.0,393.53,3.570000
273,0.22188,20.0,6.960000,1.000000,0.464,7.691,51.8,4.3665,3,223,18.6,390.77,6.580000
329,0.06724,0.0,3.240000,0.000000,0.460,6.333,17.2,5.2146,4,430,16.9,375.21,7.340000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
440,22.05110,0.0,18.100000,0.000000,0.740,5.818,92.4,1.8662,24,666,20.2,391.45,12.715432
131,1.19294,0.0,21.890000,0.000000,0.624,6.326,97.7,2.2710,4,437,21.2,396.90,12.260000
249,0.19073,22.0,5.860000,0.000000,0.431,6.718,17.5,7.8265,7,330,19.1,393.74,6.560000
152,1.12658,0.0,19.580000,0.069959,0.871,5.012,88.0,1.6102,5,403,14.7,343.28,12.120000


In [10]:
# Размер обучающей выборки
x_train.shape, y_train.shape

((354, 13), (354,))

In [11]:
# Размер тестовой выборки
x_test.shape, y_test.shape

((152, 13), (152,))

In [12]:
# Создание модели с k=5 соседями
knn_model = KNeighborsRegressor(n_neighbors=5)

# Обучение модели на обучающих данных
knn_model.fit(x_train, y_train)

# Прогнозирование на тестовых данных
y_pred = knn_model.predict(x_test)

In [16]:
# коэффициент детерминации
print(f"R^2: {r2_score(y_test, y_pred)}")
# среднеквадратичная ошибка
print(f"MSE: {mean_squared_error(y_test, y_pred)}")
# средняя абсолютная ошибка
print(f"MAE: {mean_absolute_error(y_test, y_pred)}")


R^2: 0.3944921792280742
MSE: 47.623471052631565
MAE: 5.030131578947368


In [19]:
knn_model = KNeighborsRegressor(n_neighbors=4)
# Обучение модели на обучающих данных
knn_model.fit(x_train, y_train)

# Прогнозирование на тестовых данных
y_pred = knn_model.predict(x_test)

# коэффициент детерминации
print(f"R^2: {r2_score(y_test, y_pred)}")
# среднеквадратичная ошибка
print(f"MSE: {mean_squared_error(y_test, y_pred)}")
# средняя абсолютная ошибка
print(f"MAE: {mean_absolute_error(y_test, y_pred)}")

R^2: 0.38937612423899104
MSE: 48.025851151315784
MAE: 4.977796052631579


In [20]:
knn_model = KNeighborsRegressor(n_neighbors=11)
# Обучение модели на обучающих данных
knn_model.fit(x_train, y_train)

# Прогнозирование на тестовых данных
y_pred = knn_model.predict(x_test)

# коэффициент детерминации
print(f"R^2: {r2_score(y_test, y_pred)}")
# среднеквадратичная ошибка
print(f"MSE: {mean_squared_error(y_test, y_pred)}")
# средняя абсолютная ошибка
print(f"MAE: {mean_absolute_error(y_test, y_pred)}")

R^2: 0.38314945247469157
MSE: 48.51558177468465
MAE: 5.013277511961722


## Произведите подбор гиперпараметра K с использованием GridSearchCV и RandomizedSearchCV

In [17]:
# Определение модели KNeighborsRegressor
knn = KNeighborsRegressor()

# Определение гиперпараметров, которые будут тестироваться
param_grid = {'n_neighbors': np.arange(1, 100)}

# Инициализация GridSearchCV
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='neg_mean_squared_error')

# Обучение GridSearchCV
grid_search.fit(x_train, y_train)

# Вывод лучших гиперпараметров и оценки качества
print("Лучшие гиперпараметры для GridSearchCV:", grid_search.best_params_)
print("Лучшая оценка для GridSearchCV:", -grid_search.best_score_)


Лучшие гиперпараметры для GridSearchCV: {'n_neighbors': 4}
Лучшая оценка для GridSearchCV: 46.96485802313883


In [28]:
# Инициализация RandomizedSearchCV
random_search = RandomizedSearchCV(knn, param_distributions=param_grid, n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42)

# Обучение RandomizedSearchCV
random_search.fit(x_train, y_train)

# Вывод лучших гиперпараметров и оценки качества
print("Лучшие гиперпараметры для RandomizedSearchCV:", random_search.best_params_)
print("Лучшая оценка для RandomizedSearchCV:", -random_search.best_score_)



Лучшие гиперпараметры для RandomizedSearchCV: {'n_neighbors': 11}
Лучшая оценка для RandomizedSearchCV: 53.48128490280527


In [33]:
# Оценка качества оптимальной модели с использованием кросс-валидации
kf = KFold(n_splits=5, shuffle=True, random_state=42)

grid_scores = cross_val_score(grid_search.best_estimator_, x_train, y_train, cv=kf, scoring='neg_mean_squared_error')
print("Оценка качества оптимальной модели с использованием кросс-валидации:", np.mean(-grid_scores))

random_scores = cross_val_score(random_search.best_estimator_, x_train, y_train, cv=kf, scoring='neg_mean_squared_error')
print("Оценка качества оптимальной модели с использованием кросс-валидации:", np.mean(-random_scores))

Оценка качества оптимальной модели с использованием кросс-валидации: 49.121765392354135
Оценка качества оптимальной модели с использованием кросс-валидации: 49.33459348820194
